# Load Dataset

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

,zpid,city,submarket,zipcode,address,price,like,view,duration,day,single,parking,bathroom,bedroom,age,living,description,geometry
0,4171164,CH,South,60617,10905 S Avenue H,309000,0,20,134,64,0,2.5,4.0,4.0,81.0,2376.0,Tastefully and professionally updated 4BR / 3....,POINT (-87.53252 41.69706)
1,80851025,CH,South,60616,475 W 24th St APT 5H,265000,0,44,50,7,1,1.0,2.0,2.0,19.0,900.0,Experience City living as its finest in this t...,POINT (-87.63969 41.84879)
2,2127908883,CH,West,60612,2827 W Congress Pkwy APT 3,230000,0,7,90,83,1,1.0,2.0,2.0,17.0,1300.0,"Offering exceptional comfort, convenience, and...",POINT (-87.69721 41.87457)
3,60265291,CH,North,60614,2712 N Ashland Ave #4A,460271,0,20,25,64,1,1.0,2.0,2.0,24.0,1700.0,Looking for an updated extra-wide 2 bed/2 bath...,POINT (-87.66872 41.93097)
4,3700052,CH,North,60640,5415 N Sheridan Rd APT 306,319000,2,112,37,40,1,1.0,2.0,2.0,51.0,1250.0,"Discover the allure of this 2-bed, 2-bath gem ...",POINT (-87.65439 41.97993)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10106,20383564,LA,West LA,90045,8117 Chase Ave,1811700,4,186,41,7,0,4.0,3.0,4.0,78.0,1892.0,Welcome to this impressive Westchester home vi...,POINT (-118.41125 33.96461)
10107,20510134,LA,West LA,90067,10128 Empyrean Way APT 302,2100000,0,17,178,79,1,2.0,4.0,2.0,45.0,2498.0,"From the foyer, enter to a sophisticated livin...",POINT (-118.40809 34.05322)
10108,19905411,LA,San Fernando Valley,91335,7428 Vanalden Ave,815000,0,59,5,47,0,2.0,1.0,3.0,72.0,1670.0,Major Equity opportunity in the City of Reseda...,POINT (-118.54884 34.20552)
10109,302794497,LA,West LA,90094,12678 Millennium,3715000,1,140,78,12,0,3.0,5.0,4.0,7.0,3639.0,Embrace luxury living and the best in Playa Vi...,POINT (-118.41388 33.97678)


In [ ]:
zillow_df = gpd.read_file('../dataset/raw/2. zillow_cleaned.geojson')
zillow_df

# Comparison frequency of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# --- 2. 도시별 키워드 분석 함수 ---

def get_top_keywords_by_city(df, city_name, n_top_keywords=50):
    """
    특정 도시 내에서 fast-selling 그룹과 slow-selling 그룹 간의 
    단어 빈도를 비교하여 fast-selling 그룹에서 더 중요한 키워드를 추출한다.
    """
    print(f"--- Analyzing Keywords for {city_name} ---")
    city_df = df[df['city'] == city_name].copy()
    
    if city_df.empty:
        print(f"No data found for city: {city_name}")
        return None, None

    # 그룹 분리
    fast_texts = city_df[city_df['fast_selling'] == True]['description']
    slow_texts = city_df[city_df['fast_selling'] == False]['description']

    if fast_texts.empty or slow_texts.empty:
        print(f"Not enough data for both fast and slow groups in {city_name}")
        return None, None

    # 벡터화 (CountVectorizer 사용, 필요시 옵션 조정: min_df, max_df, ngram_range, stop_words 등)
    # stop_words='english' 사용 시 일반적인 영어 불용어 제거
    # 필요하다면 NLTK 등을 사용하여 커스텀 불용어 목록 정의 가능
    vectorizer = CountVectorizer(stop_words='english', min_df=2) # 최소 2개 문서 이상 등장 단어만 고려 (예시)

    # 전체 도시 텍스트로 vocabulary 구축 후, 그룹별로 변환
    try:
        vectorizer.fit(city_df['description'])
        fast_counts = vectorizer.transform(fast_texts)
        slow_counts = vectorizer.transform(slow_texts)
    except ValueError as e:
         print(f"Vectorizer error in {city_name}: {e}. Not enough vocabulary?")
         return None, None

    vocab = vectorizer.get_feature_names_out()

    # --- 3. 단어 빈도 비교 (간단한 방법: 상대 빈도 차이) ---
    # 각 그룹 내 단어의 총 빈도 대비 상대 빈도 계산
    # (Laplace smoothing을 위해 +1 추가)
    fast_freq = (np.array(fast_counts.sum(axis=0)).flatten() + 1) / (fast_counts.sum() + len(vocab))
    slow_freq = (np.array(slow_counts.sum(axis=0)).flatten() + 1) / (slow_counts.sum() + len(vocab))

    # Fast 그룹에서 상대적으로 더 빈번한 단어 찾기 (빈도 비율 또는 차이 사용 가능)
    # 여기서는 빈도 비율 사용: fast_freq / slow_freq
    # 분모가 0이 되는 것을 방지 (위에서 +1 smoothing으로 해결됨)
    freq_ratio = fast_freq / slow_freq

    # 결과 정리
    keyword_scores = pd.DataFrame({
        'keyword': vocab,
        'fast_freq': fast_freq,
        'slow_freq': slow_freq,
        'freq_ratio': freq_ratio
    })

    # --- 4. 키워드 식별 ---
    # freq_ratio가 높을수록 fast 그룹에서 상대적으로 더 중요
    # 추가적으로 fast 그룹 내에서의 절대 빈도도 고려할 수 있음 (희귀하지만 비율만 높은 단어 제외)
    fast_term_total_counts = np.array(fast_counts.sum(axis=0)).flatten()
    keyword_scores['fast_total_count'] = fast_term_total_counts
    
    # 예: fast 그룹에서 최소 3번 이상 나타나고, 빈도 비율이 높은 순서대로 정렬
    top_keywords = keyword_scores[keyword_scores['fast_total_count'] >= 3].sort_values(by='freq_ratio', ascending=False)

    print(f"Found {len(top_keywords)} keywords significantly more frequent in fast-selling group for {city_name}.")
    
    return top_keywords.head(n_top_keywords), keyword_scores # 상위 키워드와 전체 점수 반환


In [ ]:
cities = ['NY', 'CH', 'LA']
city_keywords = {}
all_scores = {}

In [ ]:
for city in cities:
    top_k, all_s = get_top_keywords_by_city(df, city, n_top_keywords=50)
    if top_k is not None:
        city_keywords[city] = top_k
        all_scores[city] = all_s
        print(f"\nTop 5 keywords for {city}:")
        print(top_k.head())
        print("-" * 30)